In [ ]:
import tkinter as tk
from tkinter import ttk as ttk
from tkinter import messagebox as mb 
import datetime
import sqlite3
import tkcalendar


# Setting the SQL database connection
dbconnector = sqlite3.connect("Expense Tracker.db")
dbcursor = dbconnector.cursor()

dbconnector.execute("""CREATE TABLE IF NOT EXISTS expensetracker(
ID INTEGER PRIMARY KEY AUTOINCREMENT,
Payee TEXT NOT NULL,
Amount REAL NOT NULL,
ModeOfPayment TEXT NOT NULL,
Date TEXT NOT NULL,
Category TEXT NOT NULL

)""")

dbconnector.commit()

# Creating different functionalities that will
# help us to manage, edit and delete the database data
# First, a function to display all the expenses in the treeview widget:
def listallexpenses():
    global data_table, dbconnector
    data_table.delete(*data_table.get_children())
    data = dbconnector.execute('SELECT * FROM expensetracker')
    dbconnector.commit()
    # getting the data in the form of a tuple using fetchall method and storing it in variable "d"
    d = data.fetchall()
    for values in d:
        data_table.insert("","end", values=values)
    


# A function to clear the textboxes:
def clearfields():
    global data_table, payee, amount, category, modeofpayment, datefield
    confirmation = mb.askyesno("Confirmation", "Are you sure you want to clear the entry fields?")
    if confirmation:
     todaysdate = datetime.datetime.now().date()
     category.set("")
     payee.set("")
     amount.set(0.0)
     modeofpayment.set("")
     datefield.set_date(todaysdate)
     data_table.selection_remove(*data_table.selection())

# A function to delete an expense:
def deleteexpense():
    global data_table, dbconnector
    e = "No expense selected. Please select an expense to delete."
    if not data_table.selection():
        mb.showerror("Error", f"{e}")
        return
    selected_expense = data_table.item(data_table.focus())
    tuple_of_values = selected_expense["values"]
    msg = "Are you sure you want to delete this expense?"
    confirmation = mb.askyesno("Confirmation",f"{msg}")
    if confirmation:
        dbconnector.execute("DELETE FROM expensetracker WHERE id = :id", {"id" : tuple_of_values[0]})
        dbconnector.commit()
        mb.showinfo("Action Completed", "The selected record has been deleted successfully!")
        listallexpenses()

# A function to delete all expenses:
def removeallexpenses():
    global data_table, dbconnector
    msg = "Are you sure you want to delete all the expenses from records?"
    confirmation = mb.askyesno("Confirmation", f"{msg}")
    if confirmation:
        dbconnector.execute("DELETE FROM expensetracker")
        dbconnector.commit()
        mb.showinfo("Action Completed", "All the expenses have been deleted successfully!")
        listallexpenses()

# A function to Add an expense:
def addexpense():
    global data_table, payee, amount, category, modeofpayment, datefield, dbconnector
    e = "Invalid Amount."
    if float(amount.get()) < 0:
        mb.showerror("Error", f"{e}")
        return

    dbconnector.execute("INSERT INTO expensetracker(Date, Payee, Amount, Category, ModeOfPayment) VALUES (?,?,?,?,?)",
                             (datefield.get_date(), payee.get(), amount.get(), category.get(), modeofpayment.get()))
    dbconnector.commit()
    info ="Expense has been added successfully!"
    mb.showinfo("Done", f"{info}")
    # Refresh the table and clear fields
    clearfields()
    listallexpenses()


# A function to edit a field of an expense:
def editexpense():
    global data_table, payee, amount, category, modeofpayment, datefield, dbconnector
    selected_expense = data_table.item(data_table.focus())
    tuple_of_values = selected_expense["values"]
    dbconnector.execute(
        'UPDATE ExpenseTracker SET Date = ?, Payee = ?, Category = ?, Amount = ?, ModeOfPayment = ? WHERE ID = ?',
        (datefield.get_date(), payee.get(), category.get(), amount.get(), modeofpayment.get(), tuple_of_values[0])
        )
    dbconnector.commit()
    clearfields()
    listallexpenses()
    info = "Expense has been edited successfuly!"
    mb.showinfo("Done", f"{info}")

def open_new_window():
    global dbconnector, date
    new_window = tk.Toplevel() 
    new_window.title("New Window")
    new_window.resizable(0, 0)
    new_window.geometry("1100x600") 

    frame_1 = tk.Frame(new_window, background="#6D2323", padx=20, pady=10)
    frame_1.pack(side="top", fill='both', expand="yes")
    
    tk.Label(frame_1,
        text="Monthly Expense Report",
        font=("Open Sans", "20", "bold"),
        fg="#FEF9E1",
        bg="#6D2323",
        activeforeground="#FEF9E1",
        activebackground="#6D2323",
        relief="flat").pack(side="top", fill="both")
    
    frame_2 = tk.Frame(frame_1, background="#6D2323", padx=20, pady=10)
    frame_2.pack(anchor="center", fill="both", expand="yes")

    report_table = ttk.Treeview(frame_2, selectmode="browse", columns=('Date', 'Payee', 'Amount', 'ModeOfPayment', 'Category'))
    report_table.heading("Date", text="Date", anchor="center")
    report_table.heading('Payee', text='Payee', anchor="center")
    report_table.heading('Amount', text='Amount', anchor="center")
    report_table.heading('ModeOfPayment', text='Mode of Payment', anchor="center")
    report_table.heading("Category", text="Category", anchor="center")

    report_table.column('#0', width=0, stretch="no")
    report_table.column('#1', width=70, stretch="no")
    report_table.column('#2', width=100, stretch="no")
    report_table.column('#3', width=100, stretch="no")
    report_table.column('#4', width=110, stretch="no")
    report_table.column('#5', width=150, stretch="no")

    report_table.pack(fill="both", expand="yes")
    x_scrollbar_report_table = ttk.Scrollbar(report_table, orient="horizontal", command=report_table.xview)
    x_scrollbar_report_table.pack(side="bottom", fill="x")
    y_scrollbar_report_table = ttk.Scrollbar(report_table, orient="vertical", command=report_table.yview)
    y_scrollbar_report_table.pack(side="right", fill="y")

    frame_3 = tk.Frame(frame_1, bg="#FEF9E1", padx=10, pady=10)
    frame_3.pack(side="bottom", fill="both")
    tk.Label(frame_3,
        text="Summary of Expenses",
        font=("Open Sans", "20", "bold"),
        fg="#6D2323",
        bg="#FEF9E1",
        relief="flat").pack(anchor="center", fill="both")
    
    # Defining variables for latter use:
    value = 0
    avg_spending = 0
    category_amount = 0
    top_category = "N/A"
    highest_amount = 0
    highest_date = "N/A"
    highest_payee = "N/A"
    highest_category = "N/A"
    no_of_entries = 0

    # update function
    def update_report():
        global value, avg_spending, category_amount, top_category, highest_amount, highest_date, highest_payee, highest_category, no_of_entries

        today = datetime.datetime.now()
        month = today.strftime("%m")
        
        # Add data to the Treeview
        report_table.delete(*report_table.get_children())
        req_entries = dbconnector.execute("""SELECT Date, Payee, Amount, ModeOfPayment, Category FROM expensetracker
            WHERE strftime('%m', date) = ?""", (month,))
        dbconnector.commit()
        
        # Fetch data and populate the Treeview
        d = req_entries.fetchall()
        for values in d:
            report_table.insert("", "end", values=values)

        no_of_entries = len(d)

        total_amount = dbconnector.execute("""SELECT SUM(Amount) FROM expensetracker
            WHERE strftime('%m', date) = ?""", (month,))
        dbconnector.commit()
        total_amm = total_amount.fetchall()
        value = total_amm[0][0] if total_amm[0][0] else 0


        # Average Spending of the month:
        if month == '2':
            avg_spending = round(value / 28, 3)  # Assuming 30 days in the month
        elif month == '4' or month == '6' or month == '9' or month == '11':
            avg_spending = round(value/30, 3)
        else:
            avg_spending = round(value/31, 3)

        # Highest spending info
        highest_entry = dbconnector.execute("""SELECT Date, Amount, Payee, Category FROM expensetracker
            ORDER BY Amount DESC
            LIMIT 1""")
        dbconnector.commit()
        highest_entry_tup = highest_entry.fetchone()
        highest_date = highest_entry_tup[0] if highest_entry_tup else "N/A"
        highest_amount = highest_entry_tup[1] if highest_entry_tup else 0
        highest_payee = highest_entry_tup[2] if highest_entry_tup else "N/A"
        highest_category = highest_entry_tup[3] if highest_entry_tup else "N/A"

        # Top category info
        req_category = dbconnector.execute("""SELECT Category, SUM(Amount) FROM expensetracker
            GROUP BY Category
            ORDER BY SUM(Amount) DESC
            LIMIT 1""")
        req_category_tup = req_category.fetchone()
        top_category = req_category_tup[0] if req_category_tup else "N/A"
        category_amount = req_category_tup[1] if req_category_tup else 0

        message = tk.Message(frame_3, text=f"• Your total monthly expenses are: Rs.{value}\n• Your average spending of this month is: Rs.{avg_spending}\n• You spent Rs.{category_amount} on the Category: '{top_category}', which is your highest expense.\n• Your highest expense was Rs.{highest_amount} on the Date: {highest_date} to Payee: '{highest_payee}' in Category: '{highest_category}'\n• Your total number of transactions for this month is: {no_of_entries}",
            background="#FEF9E1",foreground="#6D2323", width=1000, font=("Open Sans", 15))
        message.pack(side="bottom",fill="both",expand="yes",padx=20, pady=20)

    update_report()


# Main Window
MainWindow = tk.Tk()
MainWindow.geometry("1200x600")
MainWindow.resizable(0,0)
MainWindow.title("Expense Tracker")

# Setting up a main left frame that will further house two frames
frame_left = tk.Frame(MainWindow, background="#E5D0AC", padx=20, pady=10)

# Upper left frame that will contain main title and data entry frame heading
upper_left_frame = tk.Frame(frame_left, background="#E5D0AC")
# frame1 will contain data entries and two buttons at the bottom
frame1 = tk.Frame(frame_left, background="#E5D0AC")

frame_left.pack(side="left", fill = "both")
upper_left_frame.pack(fill = "both")  
frame1.pack(fill = "both")  

#Widgets in UPPER LEFT FRAME
titlelabel = tk.Label(upper_left_frame,
    text= "EXPENSE TRACKER",
    font=("Open Sans", "17"),
    bg= "#6D2323",
    fg= "#FEF9E1",
    relief= "groove")
titlelabel.pack(fill="both")
frame1_heading = tk.Label(upper_left_frame,
    text= "DATA ENTRY FRAME",
    font=("Open Sans", "17"),
    bg= "#6D2323",
    fg= "#FEF9E1",
    relief= "groove"
    )
frame1_heading.pack(fill="both", side="bottom")

# Widgets in LOWER LEFT FRAME (frame1)
# PAYEE
payee_label =tk.Label(frame1,
    text= "Payee:",
    font= ("Open Sans", "12","bold"),
    fg= "#6D2323",
    bg="#E5D0AC"
    )
payee_label.grid(row = 1, column = 0, sticky = "w", padx = 20, pady = 10)
payee = tk.StringVar()
payee_entry = tk.Entry(frame1,
    textvariable= payee,
    font= ("Open Sans", "12"),
    bg= "#6D2323",
    fg= "#FEF9E1",
    )
payee_entry.grid(row = 1, column = 1, sticky = "w", padx = 10, pady = 10)

# AMOUNT
amount_label = tk.Label(frame1,
    text= "Amount: (in Rs.)",
    font= ("Open Sans", "12","bold"),
    bg= "#E5D0AC",
    fg= "#6D2323",
    )
amount_label.grid(row = 2, column = 0, sticky = "w", padx = 20, pady = 10)
amount = tk.DoubleVar()
amount_entry = tk.Entry(frame1,
    textvariable= amount,
    font=("Open Sans", "12"),
    fg="#FEF9E1",
    bg="#6D2323",
    relief= "groove")
amount_entry.grid(row = 2, column = 1, sticky = "w", padx = 10, pady = 10)

# MODE OF PAYMENT
mode_label = tk.Label(frame1,
    text= "Mode of Payment:",
    font= ("Open Sans", "12","bold"),
    bg= "#E5D0AC",
    fg= "#6D2323",
    )
mode_label.grid(row = 3, column = 0, sticky = "w", padx = 20, pady = 10)
modeofpayment = tk.StringVar()
mode_entry = tk.Entry(frame1,
    textvariable= modeofpayment,
    font= ("Open Sans", "12"),
    bg= "#6D2323",
    fg= "#FEF9E1",
    relief= "groove")
mode_entry.grid(row = 3, column = 1, sticky = "w", padx = 10, pady = 10)

# DATE
date_label = tk.Label(frame1,
    text= "Date:",
    font= ("Open Sans", "12","bold"),
    bg= "#E5D0AC",
    fg= "#6D2323",
    )
date_label.grid(row = 4, column = 0, sticky = "w", padx = 20, pady = 10)
datefield = tkcalendar.DateEntry(frame1, width=15)
datefield.grid(row = 4, column = 1, sticky = "w", padx = 10, pady = 10)

# CATEGORY
descrip_label = tk.Label(frame1,
    text= "Category:",
    font= ("Open Sans", "12","bold"),
    bg= "#E5D0AC",
    fg= "#6D2323",
    )
descrip_label.grid(row = 5, column = 0, sticky = "w", padx = 20, pady = 10)
category = tk.StringVar()
descrip_entry = tk.Entry(frame1,
    textvariable = category,
    font= ("Open Sans", "12"),
    bg= "#6D2323",
    fg= "#FEF9E1",
    relief= "groove")
descrip_entry.grid(row = 5, column = 1, sticky = "w", padx = 10, pady = 10)

# A FIELD ClEAR BUTTON at the bottom
field_clear_btn = tk.Button(frame1,
        text="CLEAR ENTRY FIELDS", 
        font=("Open Sans", "13", "bold"), 
        width = 20,  
        bg= "#FEF9E1",
        fg= "#6D2323",  
        activebackground = "#6D2323",  
        activeforeground = "#FEF9E1",
        borderwidth=3, highlightcolor="#FEF9E1", highlightbackground= "#FEF9E1",
        command=clearfields)
field_clear_btn.grid(row=6, column= 0, columnspan= 2,sticky ="ew", padx= 20, pady=10)

# AN ADDING EXPENSE BUTTON
add_expense_btn = tk.Button(frame1,
        text="ADD EXPENSE", 
        font=("Open Sans", "13","bold"), 
        width = 20,  
        bg= "#FEF9E1",
        fg= "#6D2323",
        activebackground = "#6D2323",  
        activeforeground = "#FEF9E1",
        borderwidth=3, highlightcolor="#FEF9E1", highlightbackground= "#FEF9E1",
        command=addexpense)
add_expense_btn.grid(row=7, column=0, columnspan=2, sticky ="ew", padx= 20, pady=10)

# Making a main RIGHT FRAME that contains an upper right frame
frame_right = tk.Frame(MainWindow,padx=20,pady=10, background="#6D2323")
frame_right.pack(side="right", fill = "both", expand="true")

# UPPER RIGHT FRAME That will consists of editing options/buttons
frame_upper_right = tk.Frame(frame_right,background="#6D2323",padx=10,pady=10 )
frame_upper_right.columnconfigure(0, weight=1)
frame_upper_right.columnconfigure(1, weight=1)
frame_upper_right.columnconfigure(2, weight=1)
frame_upper_right.rowconfigure(0,weight=1)
frame_upper_right.rowconfigure(1,weight=1)
frame_upper_right.rowconfigure(2,weight=1)

# The configure function helps in centering the buttons and have made both rows and columns get equal available space in the frame
frame_upper_right.pack(side="top", fill="both" )

# Buttons for the UPPER RIGHT FRAME
# DELETE EXPENSE BUTTON
delete_expense_btn = tk.Button(frame_upper_right,
        text="DELETE EXPENSE", 
        width=60,
        font=("Open Sans","13","bold"),   
        bg= "#FEF9E1",
        fg= "#6D2323", 
        activebackground = "#6D2323",  
        activeforeground = "#FEF9E1",
        borderwidth=3, highlightcolor="#FEF9E1", highlightbackground= "#FEF9E1",
        command=deleteexpense)
delete_expense_btn.grid(column=0, row= 0, sticky="ew", padx = 20, pady = 10,)

# EDIT EXPENSE BUTTON
edit_expense_btn = tk.Button(frame_upper_right,
        text="EDIT EXPENSE", 
        font=("Open Sans", "13","bold"), 
        width=20,
        bg= "#FEF9E1",
        fg= "#6D2323",   
        activebackground = "#6D2323",  
        activeforeground = "#FEF9E1",
        borderwidth=3, highlightcolor="#FEF9E1", highlightbackground= "#FEF9E1",
        command= editexpense)
edit_expense_btn.grid(column= 3, row= 0, sticky = "ew", padx = 20, pady = 10)

# DELETE ALL EXPENSES BUTTON
del_all_expense_button_btn =tk.Button(frame_upper_right,
        text="DELETE ALL EXPENSES", 
        font=("Open Sans", "12","bold"), 
        width=40, 
        bg= "#FEF9E1",
        fg= "#6D2323",      
        activebackground = "#6D2323",  
        activeforeground = "#FEF9E1",
        borderwidth=3, highlightcolor="#FEF9E1", highlightbackground= "#FEF9E1",
        command= removeallexpenses)
del_all_expense_button_btn.grid(column= 0, row= 3, sticky = "ew", padx = 20, pady = 10)
# DISPLAY EXPENSE BUTTON
display_expenses_btn =tk.Button(frame_upper_right,
        text="LIST ALL EXPENSES", 
        font=("Open Sans", "13","bold"),
        width=20,  
        bg= "#FEF9E1",
        fg= "#6D2323",      
        activebackground = "#6D2323",  
        activeforeground = "#FEF9E1",
        borderwidth=3, highlightcolor="#FEF9E1", highlightbackground= "#FEF9E1",
        command=listallexpenses)
display_expenses_btn.grid(row=3, column=3, padx=20, pady=10, sticky= "ew")

# DISPLAY MONTHLY EXPENSES
monthly_report_btn =tk.Button(frame_upper_right,
        text="MONTHLY REPORT", 
        width=56,
        font=("Open Sans", "13","bold"),  
        bg= "#FEF9E1",
        fg= "#6D2323",    
        activebackground = "#6D2323",  
        activeforeground = "#FEF9E1",
        borderwidth=3, highlightcolor="#FEF9E1", highlightbackground= "#FEF9E1",
        command=open_new_window
        )
monthly_report_btn.grid(row=2, column=2, padx=20, pady=10, sticky= "ew")

# Making a treeview widget in MAIN RIGHT FRAME to display expenses in the form of a table
data_table = ttk.Treeview(frame_right, selectmode= "browse", columns = ('ID', 'Payee', 'Amount', 'ModeOfPayment', 'Date', 'Category'))
data_table.heading("ID", text = "ID", anchor="center")
data_table.heading('Payee', text = 'Payee', anchor = "center")  
data_table.heading('Amount', text = 'Amount', anchor = "center")  
data_table.heading('ModeOfPayment', text = 'Mode of Payment', anchor = "center")  
data_table.heading("Date", text="Date", anchor= "center")
data_table.heading("Category", text="Category", anchor= "center")

data_table.column('#0', width = 0, stretch="no")
data_table.column('#1', width = 50, stretch = "no")  
data_table.column('#2', width = 110, stretch = "no")
data_table.column('#3', width = 130, stretch = "no")  
data_table.column('#4', width = 150, stretch = "no")  
data_table.column('#5', width = 110, stretch = "no") 
data_table.column('#6', width = 205, stretch = "no")  
data_table.pack(fill= "both", expand="true")

#Making scroll bars for better access
x_scrollbar = ttk.Scrollbar(data_table, orient= "horizontal", command= data_table.xview)
x_scrollbar.pack(side="bottom", fill= "x")
y_scrollbar = ttk.Scrollbar(data_table, orient= "vertical", command= data_table.yview)
y_scrollbar.pack(side="right", fill= "y")

MainWindow.mainloop()


C:\Users\Mariyam Zahid\AppData\Local\Temp\ipykernel_1632\505973808.py:90: DeprecationWarning: The default date adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  dbconnector.execute("INSERT INTO expensetracker(Date, Payee, Amount, Category, ModeOfPayment) VALUES (?,?,?,?,?)",
C:\Users\Mariyam Zahid\AppData\Local\Temp\ipykernel_1632\505973808.py:105: DeprecationWarning: The default date adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  dbconnector.execute(
